In [ ]:
import control as ct
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_step_response(num,den):
    ss = ct.tf2ss(num, den)
    T, yout = ct.step_response(ss)
    # print(T,yout)
    plt.figure(1)
    plt.title("System Response")
    plt.plot(T,yout)
    plt.xlabel("time[s]")
    plt.ylabel("Response")
    plt.show()


In [ ]:
def check_controllability(tf):
    """Check if system is being contrallable by controllability matrix's rank

    Parameters
    ----------
    tf : _type_
        _description_
    """
    controllability_matrix = ct.ctrb(tf.A, tf.B)
    rank = np.linalg.matrix_rank(controllability_matrix)
    if (rank == controllability_matrix.shape[0]):
        print("System is controllable")
        return True
    print("System is uncontrollable")
    return False

## Create a System and Check if Controllable or Not

In [ ]:
num = [1]
den = [1,0.1,1]
plot_step_response(num,den)
check_controllability(ct.tf2ss(num, den))
# ct.root_locus(ct.tf2ss(num, den),plot=True);

## Simulate System Response in Time Domain

In [ ]:
# Create system
sys_tf = ct.tf(num,den)
sys = ct.LinearIOSystem(sys_tf, inputs='u', outputs='y')
# Time allocation
dt = 0.1
t_0 = 0
t_end = 150
t_all = np.arange(t_0,t_end+dt,dt)
t_all
# Initial State
x = np.zeros(sys.nstates)
# u: control input (input to system)
u = 1
# Initial output
y0 = 0
# Simulation results
sim_results = [[0, u, y0]]

# Simulation Loop
for idx in range(t_all.shape[0]-1):
    T_sim = [t_all[idx], t_all[idx+1]]
    T_sim, Y_sim, X_sim = ct.input_output_response(sys,T_sim,U=u,X0=x,return_x=True)
    #print(f"T_sim:{T_sim} -- Y_sim:{Y_sim} -- X_sim:{X_sim}")
    # In reality, this will be generated by controller
    u_new = u
    sim_results.append([T_sim[-1], u_new, Y_sim[-1]])
    x = X_sim[:,1]


In [ ]:
sim_results = np.array(sim_results)
t = sim_results[:, 0]
u = sim_results[:, 1]
y = sim_results[:, 2]

# Plot inputs and outputs
plt.title("System Response")
plt.subplot(2, 1, 1)
plt.plot(t, y, 'o-')
plt.xlabel('t')
plt.ylabel('y(t)')
plt.grid()
plt.subplot(2, 1, 2)
plt.step(t, u, where='post')
plt.xlabel('t')
plt.ylabel('u(t)')
plt.grid()
plt.show()